In [385]:
import pandas as pd
import pandasql as ps
import numpy as np
from tqdm import tqdm

In [220]:
data = pd.read_csv('C:/Users/zapro/OneDrive/Desktop/test_analyst.csv', sep=';', parse_dates=['created_at'])
data

,user_id,action,created_at
0,12c0a44e92efbf27f6b22cef1d3201f6,lms_view_step_content,2017-03-01 14:41:00
1,0adca2d1d799465af7eeb297adb0c63b,lms_view_step_content,2017-03-01 17:10:00
2,bd9180dc164d6192cb6ab9b206fc8f6f,lms_view_step_content,2017-03-01 14:04:00
3,8bfa563746a579775d3a5797e8a49dd2,lms_view_step_content,2017-03-01 16:52:00
4,92c4794cb5e55b5a424ce9b19ab21961,lms_view_step_content,2017-03-01 12:01:00
...,...,...,...
56081,0fc750c381ffe5625b12563647cee9e9,lms_fin_cur_step_content,2017-03-01 16:14:00
56082,38ee61b8b18eec2ce80946762d7076c8,lms_view_step_content,2017-03-01 12:22:00
56083,264ef414e69ec46be80e624cb8099228,lms_view_step_content,2017-03-01 18:46:00
56084,51c2f5f8f3b01b9758b4b614ff60b59a,lms_fin_cur_step_content,2017-03-01 14:33:00


ПЕРВАЯ ПОПЫТКА (НЕУДАЧНАЯ)

В pandasql нет функции DATEADD, поэтому я не смог добавить внутри запроса час, как просят в условии про конец сессии. Не костылями это иначе сделать, я не знаю как :c
Возможно, я неправильно понял условие про сессию.

Тут логика такая:
    1. Берётся время первого action пользователя, если это не самостоятельная работа, как начало сессии.
    2. За конец сессии берётся время начала последнего действия перед самостоятельной работой. Может быть так, что это совпадает с первым действием. Кроме того, каждый пользователь может несколько самостоятельных делать с перерывами на практику.

Недочёт вывода в том, что время конца сессии берётся за начало последнего действия перед самостоятельной работой. Тут я в тупик зашёл, потому что пользователь может просто афкашить несколько часов, а может через минуты открыть самостоятельную работу. И тогда что брать за конец сессии? Такая же проблема и с одиночными сессиями.

In [232]:
simple_query = '''
    WITH finish_table AS (
        SELECT
            user_id, 
            action,
            action_fin, 
            fin_LEAD, 
            fin_LAG, 
            Ranks, 
            created_at,
            IIF(fin_LAG = 0, IIF(fin_LEAD = 1, IIF(action_fin = 0, 1, 0), 0), IIF(action_fin = 0, IIF(fin_LAG = 1, 2, 1), 0)) AS fin_LAG_show
        FROM
            (SELECT
                user_id,
                action,
                action_fin,
                LEAD(action_fin) OVER(PARTITION BY user_id ORDER BY created_at) AS fin_LEAD,
                LAG(action_fin) OVER(PARTITION BY user_id ORDER BY created_at) AS fin_LAG,
                Ranks,
                created_at
            FROM
                (SELECT 
                    data.user_id, 
                    data.action,
                    data.created_at,
                    IIF(data.action = "lms_fin_cur_step_content", 1, 0) AS action_fin,
                    DENSE_RANK() OVER(PARTITION BY data.user_id ORDER BY data.created_at) AS Ranks
                FROM data
                INNER JOIN 
                    (SELECT 
                        DISTINCT(user_id)
                    FROM data
                    WHERE action = "lms_fin_cur_step_content") fin_users
                ON fin_users.user_id = data.user_id) actions) finish
        WHERE fin_LAG_show in (1, 2)
        )
        
        
    SELECT
        user_id,
        "Начало сессии",
        "Конец сессии" AS "Конец сессии"
    FROM
        (SELECT
            user_id,
            CASE result_column 
                WHEN 1 THEN created_at
                WHEN 2 THEN LAG(created_at) OVER(PARTITION BY user_id ORDER BY created_at)
                WHEN 3 THEN created_at
            END AS "Начало сессии",
            CASE result_column 
                WHEN 1 THEN LEAD(created_at) OVER(PARTITION BY user_id ORDER BY created_at)
                WHEN 2 THEN created_at
                WHEN 3 THEN created_at
            END AS "Конец сессии",
            result_column
        FROM
            (SELECT
                user_id,
                action,
                created_at,
                time_LEAD,
                fin_LAG_show,
                fin_LAG,
                fin_LEAD,
                IIF(fin_LAG is NULL, IIF(time_LEAD = NULL, 3, IIF(fin_LEAD = 2, 3, 1)), IIF(fin_LAG = 1, IIF(fin_LAG_show = 2, IIF(time_LEAD is NULL, 3, 1), 2), 2)) AS result_column
            FROM
                (SELECT
                    finish_table.user_id,
                    finish_table.action,
                    finish_table.created_at,
                    LEAD(finish_table.created_at) OVER(PARTITION BY finish_table.user_id ORDER BY finish_table.created_at) AS time_LEAD,
                    finish_table.fin_LAG_show,
                    LAG(finish_table.fin_LAG_show) OVER(PARTITION BY finish_table.user_id ORDER BY finish_table.created_at) AS fin_LAG,
                    LEAD(finish_table.fin_LAG_show) OVER(PARTITION BY finish_table.user_id ORDER BY finish_table.created_at) AS fin_LEAD
                FROM
                    finish_table
                INNER JOIN
                    (SELECT 
                        user_id, 
                        COUNT(*) AS counts
                    FROM
                        finish_table
                    GROUP BY user_id
                    HAVING counts > 1) fin_t
                ON fin_t.user_id = finish_table.user_id) konec) result_table) vse_eto_tochno
    WHERE result_column in (1, 3)
    ;
    '''
data_sql = ps.sqldf(simple_query, locals())
data_sql

,user_id,Начало сессии,Конец сессии
0,0011df6282fe8cf78feeab676fbc3c27,2017-03-01 07:15:00.000000,2017-03-01 15:02:00.000000
1,00915eeb771350f3d02831fd1b412cfb,2017-03-01 16:49:00.000000,2017-03-01 18:11:00.000000
2,00915eeb771350f3d02831fd1b412cfb,2017-03-01 18:12:00.000000,2017-03-01 18:12:00.000000
3,00dbfd8e8a227ad3db71ba7b32f57fb5,2017-03-01 16:35:00.000000,2017-03-01 18:09:00.000000
4,00f5988b3068176c62639caeae323b64,2017-03-01 14:05:00.000000,2017-03-01 14:54:00.000000
...,...,...,...
1272,fefe4f594908f4f350a031bd30bc3964,2017-03-01 09:12:00.000000,2017-03-01 15:25:00.000000
1273,ff2a57cb35ad7ef29d319978a13e39e2,2017-03-01 15:38:00.000000,2017-03-01 15:57:00.000000
1274,ff31388cc384f0fc778cf336277490ce,2017-03-01 12:19:00.000000,2017-03-01 13:14:00.000000
1275,ff704c4788b2d9e807ac6bd4c78763ca,2017-03-01 15:24:00.000000,2017-03-01 15:24:00.000000


In [233]:
data_sql.to_csv('result.csv', index=False)

Вторая попытка.

К сожалению, я не смог сделать задание в 1 Sql запрос. У меня не получилось использовать циклы в запросе (pandasql не даёт (в нём отстутсвуют многие функции из sql)). 

In [436]:
simple_query = '''
    WITH table_start AS (
        SELECT 
            data.user_id, 
            data.action,
            data.created_at,
            table_3.min_time,
            LAG(data.created_at) OVER(PARTITION BY data.user_id ORDER BY data.created_at) AS time_LAG,
            IIF(data.action = 'lms_fin_cur_step_content', 1, 0) AS fin
        FROM data
        INNER JOIN
            (SELECT 
                data.user_id,
                IIF(data.action = 'lms_fin_cur_step_content', 1, 0) AS fin,
                COUNT(*) AS count,
                MIN(created_at) AS min_time
            FROM data
            INNER JOIN
                (SELECT
                    DISTINCT(user_id)
                FROM data
                WHERE action = 'lms_fin_cur_step_content'
                ) table_2
            ON table_2.user_id = data.user_id
            GROUP BY data.user_id
            HAVING count > 1
            ) table_3
        ON table_3.user_id = data.user_id
    )
    
    SELECT
        table_start.user_id,
        action,
        created_at,
        fin,
        LAG(fin) OVER(PARTITION BY user_id ORDER BY created_at) AS fin_LAG,
        LEAD(fin) OVER(PARTITION BY user_id ORDER BY created_at) AS fin_LEAD,
        time_LAG,
        min_time
    FROM 
        table_start
    ;
    '''
data_sql = ps.sqldf(simple_query, locals())
data_sql

,user_id,action,created_at,fin,fin_LAG,fin_LEAD,time_LAG,min_time
0,00063cb5da1826febf178b669eea3250,lms_fin_cur_step_content,2017-03-01 18:00:00.000000,1,NaN,1.0,None,2017-03-01 18:00:00.000000
1,00063cb5da1826febf178b669eea3250,lms_fin_cur_step_content,2017-03-01 18:07:00.000000,1,1.0,1.0,2017-03-01 18:00:00.000000,2017-03-01 18:00:00.000000
2,00063cb5da1826febf178b669eea3250,lms_fin_cur_step_content,2017-03-01 18:26:00.000000,1,1.0,1.0,2017-03-01 18:07:00.000000,2017-03-01 18:00:00.000000
3,00063cb5da1826febf178b669eea3250,lms_fin_cur_step_content,2017-03-01 18:27:00.000000,1,1.0,1.0,2017-03-01 18:26:00.000000,2017-03-01 18:00:00.000000
4,00063cb5da1826febf178b669eea3250,lms_fin_cur_step_content,2017-03-01 18:31:00.000000,1,1.0,1.0,2017-03-01 18:27:00.000000,2017-03-01 18:00:00.000000
...,...,...,...,...,...,...,...,...
35070,ffacbb7db90628bfcc8be667616dfcc7,lms_fin_cur_step_content,2017-03-01 06:41:00.000000,1,1.0,1.0,2017-03-01 06:33:00.000000,2017-03-01 06:33:00.000000
35071,ffacbb7db90628bfcc8be667616dfcc7,lms_fin_cur_step_content,2017-03-01 06:50:00.000000,1,1.0,1.0,2017-03-01 06:41:00.000000,2017-03-01 06:33:00.000000
35072,ffacbb7db90628bfcc8be667616dfcc7,lms_fin_cur_step_content,2017-03-01 06:55:00.000000,1,1.0,1.0,2017-03-01 06:50:00.000000,2017-03-01 06:33:00.000000
35073,ffacbb7db90628bfcc8be667616dfcc7,lms_fin_cur_step_content,2017-03-01 07:07:00.000000,1,1.0,1.0,2017-03-01 06:55:00.000000,2017-03-01 06:33:00.000000


In [442]:
def show_session(data):
    data.created_at = pd.to_datetime(data.created_at)
    data.min_time = pd.to_datetime(data.min_time)
    data.time_LAG = pd.to_datetime(data.time_LAG)
    start_sessions = []
    finish_sessions = []
    users_id = data.user_id.unique()
    for user_id in tqdm(users_id):
        dat = data[data.user_id == user_id]
        min_time = dat.min_time.values[0]
        for row in dat.values:
            created_at, fin, fin_LAG, fin_LEAD, time_LAG = row[2], row[3], row[4], row[5], row[6]
            start_session = min_time
            finish_session = min_time
            if (fin == 0):
                if ((created_at - min_time)/ np.timedelta64(1, 'h') < 1):
                    start_session = min_time
                elif (fin_LEAD is None):
                    finish_session = min_time + np.timedelta64(1, 'h')
                else: 
                    finish_session = min_time + np.timedelta64(1, 'h')
                    min_time = created_at
            elif (fin == 1) and (fin_LAG == 0) and ((created_at - min_time)/ np.timedelta64(1, 'h') < 1):
                finish_session = created_at
                min_time = created_at
            else:
                min_time = created_at
            start_sessions.append(start_session)
            finish_sessions.append(finish_session)
    data['start_session'] = start_sessions
    data['finish_session'] = finish_sessions
    data = data.drop_duplicates(subset=['user_id', 'start_session', 'finish_session'])[data.start_session != data.finish_session][['user_id', 'start_session', 'finish_session']].reset_index().drop(columns=['index'])
    return data

In [443]:
result = show_session(data_sql)
result

100%|█████████████████████████████████████████████████████████████████████████████| 1823/1823 [00:04<00:00, 408.44it/s]
<ipython-input-442-331ec7e8aaf8>:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data.drop_duplicates(subset=['user_id', 'start_session', 'finish_session'])[data.start_session != data.finish_session][['user_id', 'start_session', 'finish_session']].reset_index().drop(columns=['index'])


,user_id,start_session,finish_session
0,0011df6282fe8cf78feeab676fbc3c27,2017-03-01 07:15:00,2017-03-01 08:15:00
1,0011df6282fe8cf78feeab676fbc3c27,2017-03-01 14:25:00,2017-03-01 15:10:00
2,00915eeb771350f3d02831fd1b412cfb,2017-03-01 16:49:00,2017-03-01 17:49:00
3,00915eeb771350f3d02831fd1b412cfb,2017-03-01 18:11:00,2017-03-01 18:12:00
4,00915eeb771350f3d02831fd1b412cfb,2017-03-01 18:12:00,2017-03-01 18:15:00
...,...,...,...
1649,ff2a57cb35ad7ef29d319978a13e39e2,2017-03-01 15:38:00,2017-03-01 15:59:00
1650,ff31388cc384f0fc778cf336277490ce,2017-03-01 12:19:00,2017-03-01 13:15:00
1651,ff704c4788b2d9e807ac6bd4c78763ca,2017-03-01 15:24:00,2017-03-01 16:13:00
1652,ff704c4788b2d9e807ac6bd4c78763ca,2017-03-01 16:35:00,2017-03-01 17:35:00
